In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, InputLayer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import mean_tweedie_deviance
from sklearn.metrics import mean_squared_log_error

import statsmodels.api as sm

In [ ]:
data = pd.read_csv('../NormalizedWeatherDataD.csv',sep=';')
data.describe()


In [ ]:
data.shape

In [ ]:
#rows are : Date	Type de tendance barométrique	Vitesse du vent moyen 10 mn	Humidité	Variation de pression en 24 heures	Rafale sur les 10 dernières minutes	Précipitations dans les 24 dernières heures	Température (°C)	Latitude	Longitude	Altitude

#plot matrix
fig = px.scatter_matrix(data, dimensions=["Vitesse du vent moyen 10 mn", "Humidité", "Variation de pression en 24 heures", "Rafale sur les 10 dernières minutes", "Précipitations dans les 24 dernières heures", "Température (°C)"], color="Type de tendance barométrique")
#bigger
fig.update_layout(
    autosize=False,
    width=2000,
    height=2000,
)
fig.show()

In [ ]:
from plotly.subplots import make_subplots

#plot every row depending on time in separate graphs
# color depending on density
date = data['Date'].tolist()

# for i in range(1, len(data.columns)):
#     fig = go.Figure()
    
#     # Add line plot trace to subplot 1
#     fig.add_trace(go.Scatter(x=date, y=data[data.columns[i]].tolist(), mode='lines', name=data.columns[i]), row=1, col=1)
    
#     # Add density mapbox trace to subplot 2
#     fig.add_trace(go.Densitymapbox(
#         lat=data["Date"],
#         lon=data[data.columns[i]],
#         radius=10
#     ), row=1, col=2)
    
#     # Update layout
#     fig.update_layout(
#         autosize=False,
#         width=800,
#         height=300,
#         grid=dict(rows=1, columns=2, xgap=0.1, subplot_titles=('Line plot', 'Density map')),
#         mapbox=dict(
#             style='carto-positron',
#             center=dict(lon=-95, lat=38),
#             zoom=3,
#         )
#     )
    
    # fig.show()



In [ ]:
for i in range(1, len(data.columns)):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Date'], y=data
    [data.columns[i]], mode='lines', name=data.columns[i]))
    density = go.Densitymapbox(
        lat=data["Date"],
        lon=data[data.columns[i]],
        radius=10
    )
    fig.add_trace(density)
    fig.update_layout(title=data.columns[i], xaxis_title='Date', yaxis_title=data.columns[i])

    fig.show()

# données floues, séparer par zone géo ?

In [ ]:
#plot each feature relation with the temperature in different plots 
fig = px.scatter(data, x="Vitesse du vent moyen 10 mn", y="Température (°C)", color="Type de tendance barométrique")
fig.show()
fig = px.scatter(data, x="Humidité", y="Température (°C)", color="Type de tendance barométrique")
fig.show()
fig = px.scatter(data, x="Variation de pression en 24 heures", y="Température (°C)", color="Type de tendance barométrique") 
fig.show()
fig = px.scatter(data, x="Rafale sur les 10 dernières minutes", y="Température (°C)", color="Type de tendance barométrique")
fig.show()
fig = px.scatter(data, x="Précipitations dans les 24 dernières heures", y="Température (°C)", color="Type de tendance barométrique")
fig.show()




In [ ]:
#calculate the most correlated features with the temperature
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')
print(corr)
#plot the most correlated features with the temperature
fig = px.scatter(data, x="Vitesse du vent moyen 10 mn", y="Température (°C)", color="Type de tendance barométrique")
fig.show()
fig = px.scatter(data, x="Humidité", y="Température (°C)", color="Type de tendance barométrique")
fig.show()

print('the most influencial features are : Vitesse du vent moyen 10 mn and Humidité')

#plot all correlations

fig = px.imshow(corr)





# corrélations nulles

In [ ]:
#linear regression
print(data)
X = data.drop(['Température (°C)'], axis=1)
Y = data['Température (°C)']

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

# print the results
print(model.summary())


In [ ]:
# test how much a result depends on the last result time wise
# the more the result depends on the last result the more the result is time dependent
# the more the result is time dependent the more the result is not a good predictor
corr = data['Température (°C)'].corr(data['Température (°C)'].shift())
print("Correlation coefficient between consecutive temperature values:", corr)



# grosse corrélation consécutive

In [ ]:
#trying lstm
#split the data into chunks of 5 for the input and 1 for the output
#the output is the temperature of the next day

# def split_sequence(sequence, n_steps_in, n_steps_out):
#     X, y = list(), list()
#     for i in range(len(sequence)):
#         # find the end of this pattern
#         end_ix = i + n_steps_in
#         out_end_ix = end_ix + n_steps_out
#         # check if we are beyond the sequence
#         if out_end_ix > len(sequence):
#             break
#         # gather input and output parts of the pattern
#         seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix:out_end_ix, -1]
#         X.append(seq_x)
#         y.append(seq_y)
#     return np.array(X), np.array(y) 

def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

# split into samples
n_steps_in= 5

X1, y1 = df_to_X_y(Y, n_steps_in)
print(X1.shape, y1.shape)

# split into train and test sets
X_train1, y_train1 = X1[:60000], y1[:60000]
X_val1, y_val1 = X1[60000:65000], y1[60000:65000]
X_test1, y_test1 = X1[65000:], y1[65000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape


In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(5,1)))
model.add(LSTM(64))
model.add(Dense(8,'relu'))
model.add(Dense(1,'linear'))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
#edit adam learning rate ?
#default adam learning rate is 0.001
#edit adam learning rate to 0.0001 ?

cp  = ModelCheckpoint(filepath='model.h5', save_best_only=True, verbose=0)
model.fit(X_train1, y_train1, epochs=15,  validation_data=(X_test1, y_test1), callbacks=[cp])
model = keras.models.load_model('model.h5')

In [ ]:
train_predictions = model.predict(X_train1).flatten()
train_results = pd.DataFrame({'Actual': y_train1.flatten(), 'Predicted': train_predictions})
print(train_results)

#get r2 score
print('r2 score : ', r2_score(y_train1, train_predictions))

In [ ]:
#plot predictions and actual value depending on the time

fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'][:60000], y=train_results['Actual'], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=data['Date'][:60000], y=train_results['Predicted'], mode='lines', name='Predicted'))
fig.update_layout(title='Actual and Predicted values', xaxis_title='Date', yaxis_title='Temperature')
fig.show()


#plot en nuage de points avec température par le temps   
fig = px.scatter(train_results, x=data['Date'][:60000], y="Actual", color="Predicted")








In [ ]:
#same lstm network with multiple inputs
#split the data into chunks of 5 for the input and 1 for the output
# split the data into chunks of 5 for the input and 1 for the output
# the output is the temperature of the next day
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [a for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size, 7] # only use the temperature column as label
        y.append(label)
    return np.array(X), np.array(y)

# split into samples
n_steps_in= 32

X2, y2 = df_to_X_y(data, n_steps_in)
print(X2.shape, y2.shape) 


# split into train and test sets
X_train2, y_train2 = X2[:60000], y2[:60000]
X_val2, y_val2 = X2[60000:65000], y2[60000:65000]
X_test2, y_test2 = X2[65000:], y2[65000:]
print(X_train2.shape, y_train2.shape, X_val2.shape, y_val2.shape, X_test2.shape, y_test2.shape)

# define model
input_shape = (X_train2.shape[1], X_train2.shape[2])
model = Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(LSTM(units=64))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))

adamrate = 0.01
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# define callbacks
cp  = ModelCheckpoint(filepath='model2.h5', save_best_only=True, verbose=0)

# train the model
model.fit(X_train2, y_train2, epochs=20, validation_data=(X_test2, y_test2), callbacks=[cp])

# load the best model and evaluate on test set
model = keras.models.load_model('model2.h5')
test_predictions = model.predict(X_test2).flatten()
test_results = pd.DataFrame({'Actual': y_test2.flatten(), 'Predicted': test_predictions})
print(test_results)
print('r2 score : ', r2_score(y_test2, test_predictions))

In [ ]:
#output the 8 next temperatures with the temp of the 8 previous days
learning_days = 8
X3, y3 = df_to_X_y(data,learning_days*8)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)

model = Sequential()
model.add(InputLayer(input_shape=(X3_train.shape[1], X3_train.shape[2]))) #VERIFIER
model.add(LSTM(units=64))
model.add(Dense(units=8, activation='relu'))    
model.add(Dense(units=1, activation='linear'))

model.compile(optimizer='adam', loss='mse')

model.fit(X3_train, y3_train, epochs=1, batch_size=72, verbose=2)


# Generate the next X predictions
n_predictions = X[-1:]  # Use the last sequence as initial input
next_predictions = []
for _ in range(X.shape[0]):
    prediction = model.predict(n_predictions)
    next_predictions.append(prediction)
    n_predictions = np.concatenate([n_predictions[:, 1:, :], prediction[:, np.newaxis, :]], axis=1)


# Print the next X predictions
for i, prediction in enumerate(next_predictions, 1):
    print(f"Prediction {i}: {prediction[0]}")

#print r2 score